In [1]:
from pathlib import Path
import torch
from datatype_recovery.models.dataset import load_dataset_from_path


model_path = Path.home()/'trained_models/structural_4out_4hops_comp.pt'
dataset_path = Path.home()/'trainset_astera_5hops_nocomp'
dataset = load_dataset_from_path(dataset_path)

model = torch.load(model_path)

In [2]:
from torch_geometric.loader import DataLoader
import torch_geometric.transforms as T
from datatype_recovery.models.dataset.encoding import *

transform = T.Compose([ToBatchTensors()])#, ToFixedLengthTypeSeq(fixed_len=4)])
dataset.transform = transform

print(decode_typeseq(dataset[12].y))
dataset[12].y.shape

['PTR', 'STRUCT']


torch.Size([1, 22, 2])

In [3]:
model.eval()

loader = DataLoader(dataset, batch_size=1)

iterable = loader.__iter__()
data = next(iterable)

print(data.y[0].shape)
print(decode_typeseq(data.y[0]))

decode_typeseq(data.y[0][None,:,:])
data.y[0].argmax(dim=data.y[0].dim()-2)

for yy in data.y:
    print(decode_typeseq(yy))
data.y[0]

torch.Size([22, 2])
['ARR', 'uchar']
['ARR', 'uchar']


tensor([[0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 1.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [1., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.],
        [0., 0.]])

In [4]:
out = model(data.x, data.edge_index, data.batch)

print(out.shape)
for pred in out.argmax(dim=1):
    print(pred)
    print([model_type_elem_names[i] for i in pred])

out.argmax(dim=1)
out.max(dim=1).values
out[0]

torch.Size([1, 22, 4])
tensor([12, 16, 16, 16])
['uint128', 'STRUCT', 'STRUCT', 'STRUCT']


tensor([[-0.0572, -0.0158, -0.0904,  0.0697],
        [-0.0031,  0.0420,  0.0409, -0.0602],
        [-0.0453,  0.0579,  0.0026, -0.0612],
        [ 0.0658, -0.0566, -0.0207,  0.0261],
        [ 0.0212,  0.0070,  0.0036,  0.0525],
        [ 0.0640, -0.0095, -0.0410,  0.0605],
        [-0.0456,  0.0476,  0.0548, -0.0082],
        [-0.0230,  0.0271,  0.0400, -0.0312],
        [-0.0829,  0.0594,  0.0160, -0.0709],
        [ 0.0545, -0.0064, -0.0474,  0.0629],
        [-0.0919, -0.1007, -0.0173, -0.0193],
        [-0.0032,  0.0198, -0.0029, -0.0310],
        [ 0.0822, -0.0373, -0.0696,  0.0583],
        [ 0.0337, -0.0559,  0.0562,  0.0350],
        [ 0.0223, -0.0642,  0.0329, -0.0584],
        [-0.0308,  0.0056, -0.0787, -0.0398],
        [-0.0585,  0.0931,  0.0767,  0.0841],
        [-0.0352, -0.0783, -0.0459, -0.0302],
        [ 0.0491, -0.0007, -0.0167,  0.0202],
        [-0.0506, -0.0323, -0.0780, -0.0264],
        [-0.0840,  0.0506, -0.0590, -0.0418],
        [-0.0400,  0.0498, -0.0184

# Confusion Matrix
Let's build this first, then calculate F1/precision/recall

In [5]:
list((dataset_path/'processed').iterdir())

[PosixPath('/home/cls0027/trainset_astera_5hops_nocomp/processed/vars-1000_22.pt'),
 PosixPath('/home/cls0027/trainset_astera_5hops_nocomp/processed/vars-1000_19.pt'),
 PosixPath('/home/cls0027/trainset_astera_5hops_nocomp/processed/IN_MEMORY_COPY.pt'),
 PosixPath('/home/cls0027/trainset_astera_5hops_nocomp/processed/vars-1000_17.pt'),
 PosixPath('/home/cls0027/trainset_astera_5hops_nocomp/processed/vars-1000_0.pt'),
 PosixPath('/home/cls0027/trainset_astera_5hops_nocomp/processed/vars-1000_18.pt'),
 PosixPath('/home/cls0027/trainset_astera_5hops_nocomp/processed/vars-1000_13.pt'),
 PosixPath('/home/cls0027/trainset_astera_5hops_nocomp/processed/vars-1000_5.pt'),
 PosixPath('/home/cls0027/trainset_astera_5hops_nocomp/processed/vars-1000_16.pt'),
 PosixPath('/home/cls0027/trainset_astera_5hops_nocomp/processed/vars-1000_20.pt'),
 PosixPath('/home/cls0027/trainset_astera_5hops_nocomp/processed/vars-1000_15.pt'),
 PosixPath('/home/cls0027/trainset_astera_5hops_nocomp/processed/vars-1000_4

In [7]:
dataset.transform

Compose([
  ToBatchTensors()
])

In [19]:

# NOTE use this approach when doing eval

# self.max_true_seq_len = max_typesequence_len_in_dataset(dataset_path)

# if self.calc_true_acc_separately:
#     print(f'Calculating true data type accuracy with separate loader, max_seq_len={self.max_true_seq_len}')
#     ds = load_dataset_from_path(dataset_path)
#     ds.transform = T.Compose([ToBatchTensors(), ToFixedLengthTypeSeq(self.max_true_seq_len)])
#     self.truetypes_loader = DataLoader(ds, batch_size=64)

# max_typesequence_len_in_dataset(dataset_path)

4

In [24]:
FIXED_LEN = max(model.max_seq_len, max_len)
FIXED_LEN

4

In [9]:
# each row is a TRUTH class, each column is a PREDICTED class

# TODO - to build this, we need to sum the count of predictions based on
#        their true/predicted classes

# use a string key like this to ID classes and make the lookup a bit easier

# we need to know which class was true/predicted, so go ahead and convert both
# to its typeseq list form, then to a STRING KEY

valid_classes = [','.join(x) for x in TypeSequence().valid_type_sequences_for_len(4)]


predictions = {}    # pred_class: { true_class: COUNT }

# init confusion matrix
for pc in valid_classes:
    predictions[pc] = {}
    for tc in valid_classes:
        predictions[pc][f'{tc}'] = 0

invalid_pred_classes = {}   # invalid class: count

# TODO - loop through entire loader
loader = DataLoader(dataset, batch_size=1)

from datatype_recovery.models.training import partition_dataset

# train_loader, test_loader = partition_dataset(dataset, model.max_seq_len, 0.7, 64)

ctr = 0

from tqdm import tqdm
from datatype_recovery.models.metrics import accuracy, accuracy_weighted


DEVICE = 'cpu'

model.to(DEVICE)
model.eval()

USE_TQDM = True
LOADER = loader
get_data = tqdm(LOADER, total=len(LOADER)) if USE_TQDM else LOADER

correct = 0
tmp = 0

# model.to()

# HAVE TO USE A BATCH SIZE OF 1 TO EVALUATE AGAINST VAR-LENGTH SEQUENCES
# (if truth is longer than max)

for data in get_data:
    out = model(data.x.to(DEVICE), data.edge_index.to(DEVICE), data.batch.to(DEVICE))
    for i, y in enumerate(data.y):
        y_class = ','.join(decode_typeseq(y, True))
        pred_class = ','.join(decode_typeseq(out[i], force_valid_seq=True))
        predictions[pred_class][y_class] += 1

    # print(f'True class: {y_class}')
    # print(f'Predicted class: {pred_class}')
    # print(f'Raw prediction: {decode_typeseq(out)}')

    # ctr += 1
    # if ctr == 20:
    #     break

import pandas as pd

# pd.DataFrame(predictions).sum(axis=1)
df = pd.DataFrame(predictions)
df

  0%|          | 0/24691 [00:00<?, ?it/s]

 49%|████▉     | 12198/24691 [02:00<02:03, 101.20it/s]


KeyboardInterrupt: 

In [ ]:
decode_typeseq(data.y[0])

['int32', '<EMPTY>', '<EMPTY>', '<EMPTY>']

In [ ]:
pred = out
truth = data.y
pred_idxs = pred.argmax(dim=pred.dim()-2)
truth_idxs = truth.argmax(dim=truth.dim()-2)
pred_idxs
truth_idxs
decode_typeseq(truth_idxs)
decode_typeseq(out)
decode_typeseq(out, force_valid_seq=True)
decode_typeseq(data.y, force_valid_seq=True)

['PTR', 'char']

In [ ]:
from datatype_recovery.models.metrics import accuracy

#(data.y.argmax(dim=data.y.dim()-2) == out.argmax(dim=out.dim()-2)).all(dim=data.y.dim()-2).to(float)
accuracy(data.y, out)
test = out.flatten()
test

tensor([-0.0496, -0.0294, -0.0833,  0.0642,  0.0024,  0.0366,  0.0400, -0.0542,
        -0.0417,  0.0714,  0.0092, -0.0548,  0.0613, -0.0456, -0.0270,  0.0412,
         0.0239,  0.0151,  0.0005,  0.0640,  0.0635, -0.0230, -0.0300,  0.0668,
        -0.0525,  0.0443,  0.0381,  0.0031, -0.0242,  0.0342,  0.0454, -0.0263,
        -0.0868,  0.0498, -0.0010, -0.0765,  0.0518,  0.0143, -0.0400,  0.0667,
        -0.0853, -0.0952, -0.0223, -0.0085, -0.0202,  0.0230, -0.0141, -0.0275,
         0.0728, -0.0474, -0.0813,  0.0596,  0.0411, -0.0552,  0.0512,  0.0329,
         0.0230, -0.0766,  0.0284, -0.0666, -0.0227,  0.0029, -0.0756, -0.0456,
        -0.0868,  0.0846,  0.0668,  0.0915, -0.0409, -0.0930, -0.0418, -0.0408,
         0.0446,  0.0024, -0.0053,  0.0332, -0.0517, -0.0367, -0.0819, -0.0377,
        -0.0899,  0.0535, -0.0626, -0.0381, -0.0272,  0.0438, -0.0234,  0.0252,
        -0.0548, -0.0306, -0.0923,  0.0597, -0.0036,  0.0362,  0.0399, -0.0508,
        -0.0499,  0.0728,  0.0124, -0.05

In [ ]:
%timeit test.view((3, 22, 4))

2.81 µs ± 1.16 µs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [ ]:
%timeit test.reshape((3,22,4))

2.86 µs ± 226 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [ ]:
# df.loc[TRUE, PRED]
df.loc['ARR,uchar','uint128']
df['uint128'].max()

7576

In [ ]:
# index_seq = typeseq_probabilities.argmax(dim=typeseq_probabilities.dim()-2)
# out.argmax(dim=1)

pred = out
# truth = data.y[0]
truth = data.y

from datatype_recovery.models.metrics import accuracy, accuracy_weighted

accuracy(truth, pred)
accuracy_weighted(truth, pred)

tensor([0.], dtype=torch.float64)

In [ ]:
tseq = TypeSequence()
terminals = tseq.terminals
non_primitives = tseq.non_primitives
primitives = tseq.primitives
aggregates = tseq.aggregates
nonterminals = tseq.nonterminals
special = tseq.special
all_names = tseq.type_element_names

In [ ]:

terminals

['float',
 'double',
 'long double',
 'char',
 'short',
 'int32',
 'int64',
 'int128',
 'uchar',
 'ushort',
 'uint32',
 'uint64',
 'uint128',
 'void',
 'STRUCT',
 'UNION',
 'ENUM',
 'FUNC']

In [ ]:
nonterminals

['ARR', 'PTR']

In [ ]:
non_primitives

['STRUCT', 'UNION', 'FUNC', 'ARR', 'PTR', 'ARR', 'PTR']

In [ ]:
aggregates

['STRUCT', 'UNION', 'FUNC', 'ARR', 'PTR']

In [ ]:
special

[]

In [ ]:
terminals

['float',
 'double',
 'long double',
 'char',
 'short',
 'int32',
 'int64',
 'int128',
 'uchar',
 'ushort',
 'uint32',
 'uint64',
 'uint128',
 'void',
 'STRUCT',
 'UNION',
 'ENUM',
 'FUNC']

In [ ]:
# 1-level
nonterminals
single_level_count = len(terminals) + len(nonterminals)
single_level_count
len(all_names)

20

In [ ]:


for i in range(1, 5):
    print(f'{len(tseq.valid_type_sequences_for_len(i)):,} possible (valid) classes for {i}-element type sequences')

19 possible (valid) classes for 1-element type sequences
58 possible (valid) classes for 2-element type sequences
136 possible (valid) classes for 3-element type sequences
292 possible (valid) classes for 4-element type sequences


In [ ]:
[','.join(x) for x in tseq.valid_type_sequences_for_len(2)]

['float',
 'double',
 'long double',
 'char',
 'short',
 'int32',
 'int64',
 'int128',
 'uchar',
 'ushort',
 'uint32',
 'uint64',
 'uint128',
 'void',
 'PTR',
 'ARR',
 'STRUCT',
 'UNION',
 'ENUM',
 'PTR,float',
 'PTR,double',
 'PTR,long double',
 'PTR,char',
 'PTR,short',
 'PTR,int32',
 'PTR,int64',
 'PTR,int128',
 'PTR,uchar',
 'PTR,ushort',
 'PTR,uint32',
 'PTR,uint64',
 'PTR,uint128',
 'PTR,void',
 'PTR,PTR',
 'PTR,ARR',
 'PTR,STRUCT',
 'PTR,UNION',
 'PTR,ENUM',
 'PTR,FUNC',
 'ARR,float',
 'ARR,double',
 'ARR,long double',
 'ARR,char',
 'ARR,short',
 'ARR,int32',
 'ARR,int64',
 'ARR,int128',
 'ARR,uchar',
 'ARR,ushort',
 'ARR,uint32',
 'ARR,uint64',
 'ARR,uint128',
 'ARR,void',
 'ARR,PTR',
 'ARR,ARR',
 'ARR,STRUCT',
 'ARR,UNION',
 'ARR,ENUM']

In [ ]:
from varlib.datatype import *

print(ArrayType(PointerType(BuiltinType.create_void_type(), pointer_size=4), None))
[','.join(x) for x in tseq.valid_type_sequences_for_len(1)]
print(StructType(db=None,name='STRUCT'))

void*[]
STRUCT


In [ ]:
print(FunctionType(None, [], name=DataTypeCategories.Function))

(FUNC)()


In [ ]:
csv_list = [(','.join(x), str(TypeSequence.seq_to_datatype(x))) for x in tseq.valid_type_sequences_for_len(4)]
func_types = []
for entry in csv_list:
    if 'FUNC' in entry[0]:
        func_types.append(entry)
        print('\t'.join(entry))


PTR,FUNC	(*FUNC)()
PTR,PTR,FUNC	(**FUNC)()
ARR,PTR,FUNC	(*FUNC)()[]
PTR,PTR,PTR,FUNC	(***FUNC)()
PTR,ARR,PTR,FUNC	(*FUNC)()[]*
ARR,PTR,PTR,FUNC	(**FUNC)()[]
ARR,ARR,PTR,FUNC	(*FUNC)()[][]


In [ ]:
for entry in csv_list:
    print('\t'.join(entry))

float	float
double	double
long double	long double
char	char
short	short
int32	int32
int64	int64
int128	int128
uchar	uchar
ushort	ushort
uint32	uint32
uint64	uint64
uint128	uint128
void	void
PTR	*
ARR	[]
STRUCT	STRUCT
UNION	UNION
ENUM	ENUM
PTR,float	float*
PTR,double	double*
PTR,long double	long double*
PTR,char	char*
PTR,short	short*
PTR,int32	int32*
PTR,int64	int64*
PTR,int128	int128*
PTR,uchar	uchar*
PTR,ushort	ushort*
PTR,uint32	uint32*
PTR,uint64	uint64*
PTR,uint128	uint128*
PTR,void	void*
PTR,PTR	**
PTR,ARR	[]*
PTR,STRUCT	STRUCT*
PTR,UNION	UNION*
PTR,ENUM	ENUM*
PTR,FUNC	(*FUNC)()
ARR,float	float[]
ARR,double	double[]
ARR,long double	long double[]
ARR,char	char[]
ARR,short	short[]
ARR,int32	int32[]
ARR,int64	int64[]
ARR,int128	int128[]
ARR,uchar	uchar[]
ARR,ushort	ushort[]
ARR,uint32	uint32[]
ARR,uint64	uint64[]
ARR,uint128	uint128[]
ARR,void	void[]
ARR,PTR	*[]
ARR,ARR	[][]
ARR,STRUCT	STRUCT[]
ARR,UNION	UNION[]
ARR,ENUM	ENUM[]
PTR,PTR,float	float**
PTR,PTR,double	double**
PTR,PTR,l

In [ ]:
dt_list[-5]

NameError: name 'dt_list' is not defined

In [ ]:
for x in dt_list:
    print(x)

float
double
long double
char
short
int32
int64
int128
uchar
ushort
uint32
uint64
uint128
void
*
[]
STRUCT
UNION
ENUM
(*FUNC)()


In [ ]:
for i in range(3):
    print(type(i))

<class 'int'>
<class 'int'>
<class 'int'>


In [ ]:
all_classes

[['float'],
 ['double'],
 ['long double'],
 ['char'],
 ['short'],
 ['int32'],
 ['int64'],
 ['int128'],
 ['uchar'],
 ['ushort'],
 ['uint32'],
 ['uint64'],
 ['uint128'],
 ['void'],
 ['STRUCT'],
 ['UNION'],
 ['ENUM'],
 ['FUNC']]

In [ ]:
second_level_classes = []
for x in nonterminals:
    for y in all_names:
        second_level_classes.append([x, y])
second_level_classes

[['ARR', 'float'],
 ['ARR', 'double'],
 ['ARR', 'long double'],
 ['ARR', 'char'],
 ['ARR', 'short'],
 ['ARR', 'int32'],
 ['ARR', 'int64'],
 ['ARR', 'int128'],
 ['ARR', 'uchar'],
 ['ARR', 'ushort'],
 ['ARR', 'uint32'],
 ['ARR', 'uint64'],
 ['ARR', 'uint128'],
 ['ARR', 'void'],
 ['ARR', 'PTR'],
 ['ARR', 'ARR'],
 ['ARR', 'STRUCT'],
 ['ARR', 'UNION'],
 ['ARR', 'ENUM'],
 ['ARR', 'FUNC'],
 ['PTR', 'float'],
 ['PTR', 'double'],
 ['PTR', 'long double'],
 ['PTR', 'char'],
 ['PTR', 'short'],
 ['PTR', 'int32'],
 ['PTR', 'int64'],
 ['PTR', 'int128'],
 ['PTR', 'uchar'],
 ['PTR', 'ushort'],
 ['PTR', 'uint32'],
 ['PTR', 'uint64'],
 ['PTR', 'uint128'],
 ['PTR', 'void'],
 ['PTR', 'PTR'],
 ['PTR', 'ARR'],
 ['PTR', 'STRUCT'],
 ['PTR', 'UNION'],
 ['PTR', 'ENUM'],
 ['PTR', 'FUNC']]

In [ ]:
second_level_nonterminals = [x for x in second_level_classes if x[-1] not in terminals]
second_level_nonterminals

[['ARR', 'PTR'], ['ARR', 'ARR'], ['PTR', 'PTR'], ['PTR', 'ARR']]

In [ ]:
third_level_classes = []
for sln in second_level_nonterminals:
    for x in all_names:
        third_level_classes.append([*sln, x])
third_level_classes

[['ARR', 'PTR', 'float'],
 ['ARR', 'PTR', 'double'],
 ['ARR', 'PTR', 'long double'],
 ['ARR', 'PTR', 'char'],
 ['ARR', 'PTR', 'short'],
 ['ARR', 'PTR', 'int32'],
 ['ARR', 'PTR', 'int64'],
 ['ARR', 'PTR', 'int128'],
 ['ARR', 'PTR', 'uchar'],
 ['ARR', 'PTR', 'ushort'],
 ['ARR', 'PTR', 'uint32'],
 ['ARR', 'PTR', 'uint64'],
 ['ARR', 'PTR', 'uint128'],
 ['ARR', 'PTR', 'void'],
 ['ARR', 'PTR', 'PTR'],
 ['ARR', 'PTR', 'ARR'],
 ['ARR', 'PTR', 'STRUCT'],
 ['ARR', 'PTR', 'UNION'],
 ['ARR', 'PTR', 'ENUM'],
 ['ARR', 'PTR', 'FUNC'],
 ['ARR', 'ARR', 'float'],
 ['ARR', 'ARR', 'double'],
 ['ARR', 'ARR', 'long double'],
 ['ARR', 'ARR', 'char'],
 ['ARR', 'ARR', 'short'],
 ['ARR', 'ARR', 'int32'],
 ['ARR', 'ARR', 'int64'],
 ['ARR', 'ARR', 'int128'],
 ['ARR', 'ARR', 'uchar'],
 ['ARR', 'ARR', 'ushort'],
 ['ARR', 'ARR', 'uint32'],
 ['ARR', 'ARR', 'uint64'],
 ['ARR', 'ARR', 'uint128'],
 ['ARR', 'ARR', 'void'],
 ['ARR', 'ARR', 'PTR'],
 ['ARR', 'ARR', 'ARR'],
 ['ARR', 'ARR', 'STRUCT'],
 ['ARR', 'ARR', 'UNION']

In [ ]:
[x for x in third_level_classes if x[-1] not in terminals]

[['ARR', 'PTR', 'PTR'],
 ['ARR', 'PTR', 'ARR'],
 ['ARR', 'ARR', 'PTR'],
 ['ARR', 'ARR', 'ARR'],
 ['PTR', 'PTR', 'PTR'],
 ['PTR', 'PTR', 'ARR'],
 ['PTR', 'ARR', 'PTR'],
 ['PTR', 'ARR', 'ARR']]

In [ ]:
len(all_names)

20

In [ ]:
len(second_level_classes)

40

In [ ]:
len(third_level_classes)

80